:warning:**IMPORTANT NOTICE**:warning:\
*Since the initial parameterisation and gradient descent optimisation are stochastic processes, the training of a neural network is not fully reproducible.*

*Therefore, it is not recommended to re-run this script as it will overwrite the original calibration of the neural network used in the work presented here.
The purpose of this script is solely to document the training procedure and can be copied as a template to fit other new neural networks.*

*To experiment with the models calibrated here, they can be loaded from the `saved_models` directory.*

# Systematic assesment of different features used to train a ML-thermobarometer

The following sets of features are tested:

## Ti-XMg thermometer (Henry, 2005)

- Ti
- Mg
- Fe

The last two as XMg.

### Full EPMA analysis

All element easily measured by EPMA are used as input features. These are:
- Si
- Ti
- Al
- Fe
- Mn
- Mg
- Na
- K

Ca is omitted, as it can be easily measured by EPMA, but is 0 or NaN in a big part of the dataset. Can Ca not e incorporated into biotite ?!

This dataset is a bit smaller than the full dataset, as the analysis with no Na measurements hav been removed.

### Elements in biotite endmember (in PEM solid solution)

Elements which can be incorporated into biotite by an endmember substitution in the newest set of thermodynmic data are considered. This approach is used to assure a consistent set of elements in simulated biotite compositions used to prime ML models and the natural data.\
Elements with corresponding endmember substitutions are:
- Si (Tschermak substitution in Eastonite <-> Phlogopite-Annite)
- Ti (Ti substitution with Ti-biotite vie deprotonation)
- Al (Teschermak substitution in Eastonite <-> Phlogopite-Annite)
- Fe (Fe-Mg substitution in Phlogopite-Annite)
- Mg (Fe-Mg substitution in Phlogopite-Annite)
- Mn (Mn-(Mg, Fe) substitution in Phlogopite-Annite <-> Mn-biotite)

### Elements in biotite endmember (in PEM solid solution), but considering Fe2+ and Fe3+ separately
The Fe content in biotite is split into Fe2+ and Fe3+ using the estimated XFe3+ (based on the assemblage no Fe-Ox/Ilm/Mgt/Hem)

- Si
- Ti
- Al
- Fe2+
- Fe3+
- Mg
- Mn

### Elements in biotite endmember excluding Mn (FMAST)
Pre-liminary tests have shown, that Mn might be a problematic input feature. We specaulate, that high Mn in Bt is preferentially controlled by the stabillity of garnet and posibly bulks effects.\
Since there is no widely known thermobarometric system, which uses Mn in biotite, excluding it could be benificial to prevent bias towards LT/LP by minor grt-free or high-Mn samples in the training set.

- Si
- Ti
- Al
- Fe
- Mg


### Elements in biotite endmember + Stable phases

In addition to the elements in the biotite endmember, the stable phases coexisting with biotite can also be used as input features.

I restrict the stable phases to the following "index minerals" in metapelites:
- chlorite
- garnet
- cordierite
- andalusite
- staurolite
- kyanite
- sillimanite
- k-feldspar

**Additional stable phases**:

Besides the "index minerals" Fe-Ti minerals are also tested if they can be valuable input features. These are:
- ilmenite
- rutile
- titanite
- magnetite
- hematite

## Test of *P* and *T* assignments

With the data "Elements in biotite endmember" the effect of assinign *P* and *T* to the natural data is tested. The *P* and *T* assignments are done with the following methods:
- Central estimate of *P* and *T* for the corresponding MAS / Zone
- Random sampling form a uniform distribution of *P* and *T* within P-/T-uppper and lower bounds for the corresponding MAS / Zone
- Random sampling from a uniform distribution of *P* and *T* within the P-/T-upper and lower bounds and additional ordering of the T estimates after Ti-in-biotite thermometry (Henry, 2005).

## Neural network architecture

All test are performed with an identical neural network architecture.
This is done to try to isolate the effect of the input features on the performance of the neural network. However, the difference in the number of input neurons itself will already affect the network's parameterisation.

### Hidden layers
- 1 hidden layer with 16 neurons

### Activation function
- relu

### Regularisation
For these tests, a smell network is considered sufficient to avoid overfitting.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import h5py

from pathlib import Path
from keras.models import Sequential
from keras.layers import Dense, Normalization
from keras.losses import MeanSquaredError
from keras.optimizers import Adam, schedules
from keras.metrics import MeanAbsoluteError, RootMeanSquaredError
from keras.callbacks import CSVLogger, EarlyStopping
from sklearn.model_selection import train_test_split

from ml_tb.normalisation import MinMaxScaler
from ml_tb.metrics import RMSE_denormalised_P, RMSE_denormalised_T, RMSE_denormalised_temperature_only

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_6860\861062914.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import data, pre-procesing and train/test split

Load the 4 different datasets and split them into training and validation sets.

Validation set is **20%** of the training set. Approx. 200 samples.

In [2]:
VALIDATION_FRACTION = 0.2

For the scaling of the P-T data, a MinMaxScaler is used.
This scaler is defined globally and used for all datasets.

In [3]:
scaling_pt = MinMaxScaler(min=[1500, 400], max=[10000, 900], axis=0)
inv_scaling_pt = MinMaxScaler(min=[1500, 400], max=[10000, 900], axis=0, invert=True)

### **Dataset 0: Ti-XMg**

In [4]:
# load excel file
data = pd.read_excel(Path("Metapelite-Database_Bt_CLEAN_2024-02-03.xlsx"))

biotite_composition = np.zeros(shape=(len(data), 2))
biotite_composition[:, 0] = data["Bt-Ti"]
biotite_composition[:, 1] = data["Bt-XMg"]


pt = np.zeros(shape=(len(data), 2))
pt[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt[:, 1] = data["Temperature random ordered after Ti-in-Bt"]

# check for NaN values (should be already filtered out)
print("NaN values in biotite composition: ", np.isnan(biotite_composition).any())
print("NaN values in PT: ", np.isnan(pt).any())

NaN values in biotite composition:  False
NaN values in PT:  False


In [5]:
# test train split
DATA00_biotite_composition_train, DATA00_biotite_composition_val, DATA00_pt_train, DATA00_pt_val = train_test_split(biotite_composition, pt, test_size=VALIDATION_FRACTION, shuffle=True)

DATA00_normalisation_biotite_composition = Normalization(axis=-1)
DATA00_normalisation_biotite_composition.adapt(DATA00_biotite_composition_train)

print(DATA00_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(DATA00_normalisation_biotite_composition.variance.numpy()))

DATA00_pt_train_norm = scaling_pt(DATA00_pt_train)
DATA00_pt_val_norm = scaling_pt(DATA00_pt_val)

print("After normalisation, the minimal value of P and T is: ", DATA00_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", DATA00_pt_train_norm.numpy().max(axis=0))

[[0.12074334 0.49643874]]
[[0.04852281 0.10346945]]
After normalisation, the minimal value of P and T is:  [0.00218576 0.00066034]
After normalisation, the maximal value of P and T is:  [0.940462  0.8988128]


### **Dataset 1: Full EPMA analysis**

In [6]:
# load excel file
data = pd.read_excel(Path("Metapelite-Database_Bt_CLEAN_InclBt-Na2024-02-16.xlsx"))

biotite_composition = np.zeros(shape=(len(data), 8))
biotite_composition[:, 0] = data["Bt-Si"]
biotite_composition[:, 1] = data["Bt-Ti"]
biotite_composition[:, 2] = data["Bt-Al"]
biotite_composition[:, 3] = data["Bt-FeTot"]
biotite_composition[:, 4] = data["Bt-Mn"]
biotite_composition[:, 5] = data["Bt-Mg"]
biotite_composition[:, 6] = data["Bt-K"]
biotite_composition[:, 7] = data["Bt-Na"]


pt = np.zeros(shape=(len(data), 2))
pt[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt[:, 1] = data["Temperature random ordered after Ti-in-Bt"]

# check for NaN values (should be already filtered out)
print("NaN values in biotite composition: ", np.isnan(biotite_composition).any())
print("NaN values in PT: ", np.isnan(pt).any())

NaN values in biotite composition:  False
NaN values in PT:  False


In [7]:
# test train split
DATA01_biotite_composition_train, DATA01_biotite_composition_val, DATA01_pt_train, DATA01_pt_val = train_test_split(biotite_composition, pt, test_size=VALIDATION_FRACTION, shuffle=True)

DATA01_normalisation_biotite_composition = Normalization(axis=-1)
DATA01_normalisation_biotite_composition.adapt(DATA01_biotite_composition_train)

print(DATA01_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(DATA01_normalisation_biotite_composition.variance.numpy()))

DATA01_pt_train_norm = scaling_pt(DATA01_pt_train)
DATA01_pt_val_norm = scaling_pt(DATA01_pt_val)

print("After normalisation, the minimal value of P and T is: ", DATA01_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", DATA01_pt_train_norm.numpy().max(axis=0))

[[2.7067592  0.12068279 1.7125479  1.229531   0.00839963 1.0890671
  0.8593252  0.03174109]]
[[0.04788255 0.04785116 0.09206234 0.23546022 0.00729814 0.26658195
  0.05551824 0.01666494]]
After normalisation, the minimal value of P and T is:  [0.00225732 0.00789404]
After normalisation, the maximal value of P and T is:  [0.9095139 0.8900758]


### **Dataset 2: Elements in biotite solution model**


In [8]:
# load excel file
data = pd.read_excel(Path("Metapelite-Database_Bt_CLEAN_2024-02-03.xlsx"))

biotite_composition = np.zeros(shape=(len(data), 6))
biotite_composition[:, 0] = data["Bt-Si"]
biotite_composition[:, 1] = data["Bt-Ti"]
biotite_composition[:, 2] = data["Bt-Al"]
biotite_composition[:, 3] = data["Bt-FeTot"]
biotite_composition[:, 4] = data["Bt-Mn"]
biotite_composition[:, 5] = data["Bt-Mg"]

pt = np.zeros(shape=(len(data), 2))
pt[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt[:, 1] = data["Temperature random ordered after Ti-in-Bt"]

# check for NaN values (should be already filtered out)
print("NaN values in biotite composition: ", np.isnan(biotite_composition).any())
print("NaN values in PT: ", np.isnan(pt).any())

NaN values in biotite composition:  False
NaN values in PT:  False


In [9]:
# test train split
DATA02_biotite_composition_train, DATA02_biotite_composition_val, DATA02_pt_train, DATA02_pt_val = train_test_split(biotite_composition, pt, test_size=VALIDATION_FRACTION, shuffle=True)

DATA02_normalisation_biotite_composition = Normalization(axis=-1)
DATA02_normalisation_biotite_composition.adapt(DATA02_biotite_composition_train)

print(DATA02_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(DATA02_normalisation_biotite_composition.variance.numpy()))

DATA02_pt_train_norm = scaling_pt(DATA02_pt_train)
DATA02_pt_val_norm = scaling_pt(DATA02_pt_val)

print("After normalisation, the minimal value of P and T is: ", DATA02_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", DATA02_pt_train_norm.numpy().max(axis=0))

[[2.7070892  0.12034276 1.7112563  1.2297457  0.0085348  1.0907588 ]]
[[0.0470589  0.04717307 0.09299814 0.23349644 0.0073014  0.26396573]]
After normalisation, the minimal value of P and T is:  [0.00010304 0.00606122]
After normalisation, the maximal value of P and T is:  [0.940462  0.8988128]


### **Dataset 3: Elements in biotite solution model (Fe2+ and Fe3+)**

In [10]:
# load excel file
data = pd.read_excel(Path("Metapelite-Database_Bt_CLEAN_2024-02-03.xlsx"))

biotite_composition = np.zeros(shape=(len(data), 7))
biotite_composition[:, 0] = data["Bt-Si"]
biotite_composition[:, 1] = data["Bt-Ti"]
biotite_composition[:, 2] = data["Bt-Al"]
biotite_composition[:, 3] = (1-data["Bt-XFe3+"]) * data["Bt-FeTot"]
biotite_composition[:, 4] = data["Bt-XFe3+"] * data["Bt-FeTot"]
biotite_composition[:, 5] = data["Bt-Mn"]
biotite_composition[:, 6] = data["Bt-Mg"]

pt = np.zeros(shape=(len(data), 2))
pt[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt[:, 1] = data["Temperature random ordered after Ti-in-Bt"]

# check for NaN values (should be already filtered out)
print("NaN values in biotite composition: ", np.isnan(biotite_composition).any())
print("NaN values in PT: ", np.isnan(pt).any())

NaN values in biotite composition:  False
NaN values in PT:  False


In [11]:
# test train split
DATA03_biotite_composition_train, DATA03_biotite_composition_val, DATA03_pt_train, DATA03_pt_val = train_test_split(biotite_composition, pt, test_size=VALIDATION_FRACTION, shuffle=True)

DATA03_normalisation_biotite_composition = Normalization(axis=-1)
DATA03_normalisation_biotite_composition.adapt(DATA03_biotite_composition_train)

print(DATA03_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(DATA03_normalisation_biotite_composition.variance.numpy()))

DATA03_pt_train_norm = scaling_pt(DATA03_pt_train)
DATA03_pt_val_norm = scaling_pt(DATA03_pt_val)

print("After normalisation, the minimal value of P and T is: ", DATA03_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", DATA03_pt_train_norm.numpy().max(axis=0))

[[2.7073662  0.1202416  1.7113693  1.0929346  0.13681658 0.00848108
  1.0903707 ]]
[[0.04788665 0.04813645 0.09521717 0.20715769 0.03111816 0.00743915
  0.26727816]]
After normalisation, the minimal value of P and T is:  [0.00010304 0.00066034]
After normalisation, the maximal value of P and T is:  [0.92656535 0.8988128 ]


### **Dataset 4: PT-sensitive elements in Biotite**

In [12]:
# load excel file
data = pd.read_excel(Path("Metapelite-Database_Bt_CLEAN_2024-02-03.xlsx"))

biotite_composition = np.zeros(shape=(len(data), 6))
biotite_composition[:, 0] = data["Bt-Si"]
biotite_composition[:, 1] = data["Bt-Ti"]
biotite_composition[:, 2] = data["Bt-Al"]
biotite_composition[:, 3] = data["Bt-FeTot"]
biotite_composition[:, 4] = data["Bt-Mn"]
biotite_composition[:, 5] = data["Bt-Mg"]

pt = np.zeros(shape=(len(data), 2))
pt[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt[:, 1] = data["Temperature random ordered after Ti-in-Bt"]

# check for NaN values (should be already filtered out)
print("NaN values in biotite composition: ", np.isnan(biotite_composition).any())
print("NaN values in PT: ", np.isnan(pt).any())

NaN values in biotite composition:  False
NaN values in PT:  False


In [13]:
# test train split
DATA04_biotite_composition_train, DATA04_biotite_composition_val, DATA04_pt_train, DATA04_pt_val = train_test_split(biotite_composition, pt, test_size=VALIDATION_FRACTION, shuffle=True)

DATA04_normalisation_biotite_composition = Normalization(axis=-1)
DATA04_normalisation_biotite_composition.adapt(DATA04_biotite_composition_train)

print(DATA04_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(DATA04_normalisation_biotite_composition.variance.numpy()))

DATA04_pt_train_norm = scaling_pt(DATA04_pt_train)
DATA04_pt_val_norm = scaling_pt(DATA04_pt_val)

print("After normalisation, the minimal value of P and T is: ", DATA04_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", DATA04_pt_train_norm.numpy().max(axis=0))

[[2.707142   0.11993986 1.7121568  1.2327652  0.00843049 1.0877595 ]]
[[0.04660965 0.04695587 0.09033892 0.22459745 0.00733271 0.25385737]]
After normalisation, the minimal value of P and T is:  [0.004532   0.00066034]
After normalisation, the maximal value of P and T is:  [0.92656535 0.8988128 ]


### **Dataset 5: Elements in biotite solution model + stable phases**

In [14]:
# load excel file
data = pd.read_excel(Path("Metapelite-Database_Bt_CLEAN_2024-02-03.xlsx"))

biotite_composition = np.zeros(shape=(len(data), 6))
biotite_composition[:, 0] = data["Bt-Si"]
biotite_composition[:, 1] = data["Bt-Ti"]
biotite_composition[:, 2] = data["Bt-Al"]
biotite_composition[:, 3] = data["Bt-FeTot"]
biotite_composition[:, 4] = data["Bt-Mn"]
biotite_composition[:, 5] = data["Bt-Mg"]

# extract one-hot encoded minerals in the following order: Chl, Grt, Crd, And, St, Ky, Sil, Kfs
index_minerals = np.zeros(shape=(len(data), 8))
index_minerals[:, 0] = data["Chl"]
index_minerals[:, 1] = data["Grt"]
index_minerals[:, 2] = data["Crd"]
index_minerals[:, 3] = data["And"]
index_minerals[:, 4] = data["St"]
index_minerals[:, 5] = data["Ky"]
index_minerals[:, 6] = data["Sil"]
index_minerals[:, 7] = data["Kfs"]

# Some minerals (Chl, Grt, St) have NaN values. Replace them with 0. Most likely samples with regional or metastable phases?!
index_minerals = np.nan_to_num(index_minerals, nan=0)

# combine biotite composition and one-hot encoded minerals
biotite_composition_idxmin = np.concatenate((biotite_composition, index_minerals), axis=1)


pt = np.zeros(shape=(len(data), 2))
pt[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt[:, 1] = data["Temperature random ordered after Ti-in-Bt"]

# check for NaN values (should be already filtered out)
print("NaN values in biotite composition & index minerals: ", np.isnan(biotite_composition_idxmin).any())
print("NaN values in PT: ", np.isnan(pt).any())

NaN values in biotite composition & index minerals:  False
NaN values in PT:  False


In [15]:
# test train split
DATA05_biotite_composition_train, DATA05_biotite_composition_val, DATA05_pt_train, DATA05_pt_val = train_test_split(biotite_composition_idxmin, pt, test_size=VALIDATION_FRACTION, shuffle=True)

DATA05_normalisation_biotite_composition = Normalization(axis=-1)
DATA05_normalisation_biotite_composition.adapt(DATA05_biotite_composition_train)

# the one-hot encoded minerals should not be normalised. This is not necessary, as they are already in the range [0, 1]
# Set the mean and variance to 0 and 1, respectively
norm_mean = DATA05_normalisation_biotite_composition.mean.numpy()
norm_mean[0,-8:] = 0
norm_var = DATA05_normalisation_biotite_composition.variance.numpy()
norm_var[0,-8:] = 1
DATA05_normalisation_biotite_composition.mean = tf.convert_to_tensor(norm_mean)
DATA05_normalisation_biotite_composition.variance = tf.convert_to_tensor(norm_var)

print(DATA05_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(DATA05_normalisation_biotite_composition.variance.numpy()))

DATA05_pt_train_norm = scaling_pt(DATA05_pt_train)
DATA05_pt_val_norm = scaling_pt(DATA05_pt_val)

print("After normalisation, the minimal value of P and T is: ", DATA05_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", DATA05_pt_train_norm.numpy().max(axis=0))

[[2.7069569  0.12051728 1.7111886  1.2338823  0.00840485 1.0865856
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]
[[0.04620519 0.04797825 0.09186491 0.22644351 0.00732265 0.2579206
  1.         1.         1.         1.         1.         1.
  1.         1.        ]]
After normalisation, the minimal value of P and T is:  [0.00010304 0.00066034]
After normalisation, the maximal value of P and T is:  [0.940462  0.8988128]


### **Dataset 6: Elements in biotite solution model + stable phases + Fe-Ti minerals**

In [16]:
# load excel file
data = pd.read_excel(Path("Metapelite-Database_Bt_CLEAN_2024-02-03.xlsx"))

biotite_composition = np.zeros(shape=(len(data), 6))
biotite_composition[:, 0] = data["Bt-Si"]
biotite_composition[:, 1] = data["Bt-Ti"]
biotite_composition[:, 2] = data["Bt-Al"]
biotite_composition[:, 3] = data["Bt-FeTot"]
biotite_composition[:, 4] = data["Bt-Mn"]
biotite_composition[:, 5] = data["Bt-Mg"]

# extract one-hot encoded minerals in the following order: Chl, Grt, Crd, And, St, Ky, Sil, Kfs + Ilm, Rt, Ttn, Mgt, Hem
index_minerals = np.zeros(shape=(len(data), 13))
index_minerals[:, 0] = data["Chl"]
index_minerals[:, 1] = data["Grt"]
index_minerals[:, 2] = data["Crd"]
index_minerals[:, 3] = data["And"]
index_minerals[:, 4] = data["St"]
index_minerals[:, 5] = data["Ky"]
index_minerals[:, 6] = data["Sil"]
index_minerals[:, 7] = data["Kfs"]
index_minerals[:, 8] = data["Ilm"]
index_minerals[:, 9] = data["Rt"]
index_minerals[:, 10] = data["Ttn"]
index_minerals[:, 11] = data["Mag"]
index_minerals[:, 12] = data["Hem"]

# Some minerals (Chl, Grt, St) have NaN values. Replace them with 0. Most likely samples with regional or metastable phases?!
index_minerals = np.nan_to_num(index_minerals, nan=0)

# combine biotite composition and one-hot encoded minerals
biotite_composition_idxmin = np.concatenate((biotite_composition, index_minerals), axis=1)


pt = np.zeros(shape=(len(data), 2))
pt[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt[:, 1] = data["Temperature random ordered after Ti-in-Bt"]

# check for NaN values (should be already filtered out)
print("NaN values in biotite composition & index minerals: ", np.isnan(biotite_composition_idxmin).any())
print("NaN values in PT: ", np.isnan(pt).any())

NaN values in biotite composition & index minerals:  False
NaN values in PT:  False


In [17]:
# test train split
DATA06_biotite_composition_train, DATA06_biotite_composition_val, DATA06_pt_train, DATA06_pt_val = train_test_split(biotite_composition_idxmin, pt, test_size=VALIDATION_FRACTION, shuffle=True)

DATA06_normalisation_biotite_composition = Normalization(axis=-1)
DATA06_normalisation_biotite_composition.adapt(DATA06_biotite_composition_train)

# the one-hot encoded minerals should not be normalised. This is not necessary, as they are already in the range [0, 1]
# Set the mean and variance to 0 and 1, respectively
norm_mean = DATA06_normalisation_biotite_composition.mean.numpy()
norm_mean[0,6:] = 0
norm_var = DATA06_normalisation_biotite_composition.variance.numpy()
norm_var[0,6:] = 1
DATA06_normalisation_biotite_composition.mean = tf.convert_to_tensor(norm_mean)
DATA06_normalisation_biotite_composition.variance = tf.convert_to_tensor(norm_var)

print(DATA06_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(DATA06_normalisation_biotite_composition.variance.numpy()))

DATA06_pt_train_norm = scaling_pt(DATA06_pt_train)
DATA06_pt_val_norm = scaling_pt(DATA06_pt_val)

print("After normalisation, the minimal value of P and T is: ", DATA06_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", DATA06_pt_train_norm.numpy().max(axis=0))

[[2.7071054  0.12190356 1.7116615  1.22817    0.00847601 1.088991
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]]
[[0.04697141 0.0480523  0.09048265 0.22438097 0.00741089 0.25464892
  1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.        ]]
After normalisation, the minimal value of P and T is:  [0.00010304 0.00066034]
After normalisation, the maximal value of P and T is:  [0.940462  0.8939979]


## Set-up global training parameters

Define a function to calculate RMSE for pressure and temperature for unscaled values to have an interpretable metric.

All models are trained for a maximum of 5000 epochs.\
Early stopping is used with a patience (no improvement of val loss) of 50 epochs.\
Inverse time learning rate decay is used for all models.

In [18]:
def RMSE_P(y_true, y_pred):
    return RMSE_denormalised_P(y_true, y_pred, inv_scaling_pt)


def RMSE_T(y_true, y_pred):
    return RMSE_denormalised_T(y_true, y_pred, inv_scaling_pt)

In [19]:
BATCH_SIZE = 50
STEPS_PER_EPOCH = len(DATA02_biotite_composition_train) // BATCH_SIZE
MAX_EPOCHS = 5000

lr_schedule = schedules.InverseTimeDecay(0.001, decay_steps=STEPS_PER_EPOCH*1000, decay_rate=1, staircase=False)

LOSS = MeanSquaredError()
METRICS = [MeanAbsoluteError(), RootMeanSquaredError(), RMSE_P, RMSE_T]

## **Test 00:** Ti-XMg thermometer

In [20]:
CALLBACKS_00 = [CSVLogger("Features00_TiXMg.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [21]:
OPT = Adam(lr_schedule)

model_00 = Sequential()
model_00.add(DATA00_normalisation_biotite_composition)
model_00.add(Dense(16, activation="relu"))
model_00.add(Dense(2))

model_00.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
model_00.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 2)                 5         
 on)                                                             
                                                                 
 dense (Dense)               (None, 16)                48        
                                                                 
 dense_1 (Dense)             (None, 2)                 34        
                                                                 
Total params: 87 (352.00 Byte)
Trainable params: 82 (328.00 Byte)
Non-trainable params: 5 (24.00 Byte)
_________________________________________________________________


In [22]:
history = model_00.fit(DATA00_biotite_composition_train, DATA00_pt_train_norm,
                       batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                       validation_data=[DATA00_biotite_composition_val, DATA00_pt_val_norm],
                       callbacks=CALLBACKS_00, verbose=False)
model_00.save(Path("saved_models", "model_feature_set_00"))



INFO:tensorflow:Assets written to: saved_models\model_feature_set_00\assets


INFO:tensorflow:Assets written to: saved_models\model_feature_set_00\assets


## **Test 01:** Full EPMA analysis

In [23]:
CALLBACKS_01 = [CSVLogger("Features01_full_EPMA_analysis.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [24]:
OPT = Adam(lr_schedule)

model_01 = Sequential()
model_01.add(DATA01_normalisation_biotite_composition)
model_01.add(Dense(16, activation="relu"))
model_01.add(Dense(2))

model_01.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
model_01.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 8)                 17        
 tion)                                                           
                                                                 
 dense_2 (Dense)             (None, 16)                144       
                                                                 
 dense_3 (Dense)             (None, 2)                 34        
                                                                 
Total params: 195 (784.00 Byte)
Trainable params: 178 (712.00 Byte)
Non-trainable params: 17 (72.00 Byte)
_________________________________________________________________


In [25]:
history = model_01.fit(DATA01_biotite_composition_train, DATA01_pt_train_norm,
                       batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                       validation_data=[DATA01_biotite_composition_val, DATA01_pt_val_norm],
                       callbacks=CALLBACKS_01, verbose=False)
model_01.save(Path("saved_models", "model_feature_set_01"))

INFO:tensorflow:Assets written to: saved_models\model_feature_set_01\assets


INFO:tensorflow:Assets written to: saved_models\model_feature_set_01\assets


## **Test 02:** Elements in biotite solution model

In [26]:
CALLBACKS_02 = [CSVLogger("Features02_elements_solution_model.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [27]:
OPT = Adam(lr_schedule)

model_02 = Sequential()
model_02.add(DATA02_normalisation_biotite_composition)
model_02.add(Dense(16, activation="relu"))
model_02.add(Dense(2))

model_02.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
model_02.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_2 (Normaliza  (None, 6)                 13        
 tion)                                                           
                                                                 
 dense_4 (Dense)             (None, 16)                112       
                                                                 
 dense_5 (Dense)             (None, 2)                 34        
                                                                 
Total params: 159 (640.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


In [28]:
history = model_02.fit(DATA02_biotite_composition_train, DATA02_pt_train_norm,
                       batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                       validation_data=[DATA02_biotite_composition_val, DATA02_pt_val_norm],
                       callbacks=CALLBACKS_02, verbose=False)
model_02.save(Path("saved_models", "model_feature_set_02"))

INFO:tensorflow:Assets written to: saved_models\model_feature_set_02\assets


INFO:tensorflow:Assets written to: saved_models\model_feature_set_02\assets


## **Test 03:** Elements in biotite solution model (Fe2+ and Fe3+)

In [29]:
CALLBACKS_03 = [CSVLogger("Features03_elements_solution_model_Fe2Fe3.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [30]:
OPT = Adam(lr_schedule)

model_03 = Sequential()
model_03.add(DATA03_normalisation_biotite_composition)
model_03.add(Dense(16, activation="relu"))
model_03.add(Dense(2))

model_03.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
model_03.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_3 (Normaliza  (None, 7)                 15        
 tion)                                                           
                                                                 
 dense_6 (Dense)             (None, 16)                128       
                                                                 
 dense_7 (Dense)             (None, 2)                 34        
                                                                 
Total params: 177 (712.00 Byte)
Trainable params: 162 (648.00 Byte)
Non-trainable params: 15 (64.00 Byte)
_________________________________________________________________


In [31]:
history = model_03.fit(DATA03_biotite_composition_train, DATA03_pt_train_norm,
                       batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                       validation_data=[DATA03_biotite_composition_val, DATA03_pt_val_norm],
                       callbacks=CALLBACKS_03, verbose=False)
model_03.save(Path("saved_models", "model_feature_set_03"))

INFO:tensorflow:Assets written to: saved_models\model_feature_set_03\assets


INFO:tensorflow:Assets written to: saved_models\model_feature_set_03\assets


## **Test 04:** Elements in biotite solution model, excluding Mn

In [32]:
CALLBACKS_04 = [CSVLogger("Features04_elements_solution_model_exclMn.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [33]:
OPT = Adam(lr_schedule)

model_04 = Sequential()
model_04.add(DATA04_normalisation_biotite_composition)
model_04.add(Dense(16, activation="relu"))
model_04.add(Dense(2))

model_04.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
model_04.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_4 (Normaliza  (None, 6)                 13        
 tion)                                                           
                                                                 
 dense_8 (Dense)             (None, 16)                112       
                                                                 
 dense_9 (Dense)             (None, 2)                 34        
                                                                 
Total params: 159 (640.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


In [34]:
history = model_04.fit(DATA04_biotite_composition_train, DATA04_pt_train_norm,
                       batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                       validation_data=[DATA04_biotite_composition_val, DATA04_pt_val_norm],
                       callbacks=CALLBACKS_04, verbose=False)
model_04.save(Path("saved_models", "model_feature_set_04"))

INFO:tensorflow:Assets written to: saved_models\model_feature_set_04\assets


INFO:tensorflow:Assets written to: saved_models\model_feature_set_04\assets


## **Test 05:** Elements in biotite solution model + stable phases

In [35]:
CALLBACKS_05 = [CSVLogger("Features05_Bt_comp_index_min.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [36]:
OPT = Adam(lr_schedule)

model_05 = Sequential()
model_05.add(DATA05_normalisation_biotite_composition)
model_05.add(Dense(16, activation="relu"))
model_05.add(Dense(2))

model_05.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
model_05.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_5 (Normaliza  (None, 14)                29        
 tion)                                                           
                                                                 
 dense_10 (Dense)            (None, 16)                240       
                                                                 
 dense_11 (Dense)            (None, 2)                 34        
                                                                 
Total params: 303 (1.19 KB)
Trainable params: 274 (1.07 KB)
Non-trainable params: 29 (120.00 Byte)
_________________________________________________________________


In [37]:
history = model_05.fit(DATA05_biotite_composition_train, DATA05_pt_train_norm,
                       batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                       validation_data=[DATA05_biotite_composition_val, DATA05_pt_val_norm],
                       callbacks=CALLBACKS_05, verbose=False)
model_05.save(Path("saved_models", "model_feature_set_05"))

INFO:tensorflow:Assets written to: saved_models\model_feature_set_05\assets


INFO:tensorflow:Assets written to: saved_models\model_feature_set_05\assets


## **Test 06:** Elements in biotite solution model + stable phases + Fe-Ti minerals

In [38]:
CALLBACKS_06 = [CSVLogger("Features06_Bt_comp_index_min.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [39]:
OPT = Adam(lr_schedule)

model_06 = Sequential()
model_06.add(DATA06_normalisation_biotite_composition)
model_06.add(Dense(16, activation="relu"))
model_06.add(Dense(2))

model_06.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
model_06.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_6 (Normaliza  (None, 19)                39        
 tion)                                                           
                                                                 
 dense_12 (Dense)            (None, 16)                320       
                                                                 
 dense_13 (Dense)            (None, 2)                 34        
                                                                 
Total params: 393 (1.54 KB)
Trainable params: 354 (1.38 KB)
Non-trainable params: 39 (160.00 Byte)
_________________________________________________________________


In [40]:
history = model_06.fit(DATA06_biotite_composition_train, DATA06_pt_train_norm,
                       batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                       validation_data=[DATA06_biotite_composition_val, DATA06_pt_val_norm],
                       callbacks=CALLBACKS_06, verbose=False)

model_06.save(Path("saved_models", "model_feature_set_06"))

INFO:tensorflow:Assets written to: saved_models\model_feature_set_06\assets


INFO:tensorflow:Assets written to: saved_models\model_feature_set_06\assets


# **Test different ways to assign temperature and pressure to the samples**

Reload dataset 02:

In [41]:
# load excel file
data = pd.read_excel(Path("Metapelite-Database_Bt_CLEAN_2024-02-03.xlsx"))

biotite_composition = np.zeros(shape=(len(data), 6))
biotite_composition[:, 0] = data["Bt-Si"]
biotite_composition[:, 1] = data["Bt-Ti"]
biotite_composition[:, 2] = data["Bt-Al"]
biotite_composition[:, 3] = data["Bt-FeTot"]
biotite_composition[:, 4] = data["Bt-Mn"]
biotite_composition[:, 5] = data["Bt-Mg"]

# check for NaN values (should be already filtered out)
print("NaN values in biotite composition: ", np.isnan(biotite_composition).any())

NaN values in biotite composition:  False


In [42]:
pt_central = np.zeros(shape=(len(data), 2))
pt_central[:, 0] = data["Pressure estimate center"] * 1000 # convert to bar
pt_central[:, 1] = data["Temperature estimate center"]

print("NaN values in PT: ", np.isnan(pt_central).any())

# train test split
PT01_biotite_composition_train, PT01_biotite_composition_val, PT01_pt_train, PT01_pt_val = train_test_split(biotite_composition, pt_central, test_size=VALIDATION_FRACTION, shuffle=True)

PT01_normalisation_biotite_composition = Normalization(axis=-1)
PT01_normalisation_biotite_composition.adapt(PT01_biotite_composition_train)

print(PT01_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(PT01_normalisation_biotite_composition.variance.numpy()))

PT01_pt_train_norm = scaling_pt(PT01_pt_train)
PT01_pt_val_norm = scaling_pt(PT01_pt_val)

print("After normalisation, the minimal value of P and T is: ", PT01_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", PT01_pt_train_norm.numpy().max(axis=0))

NaN values in PT:  False
[[2.7085605  0.11989018 1.7098479  1.2271507  0.00851352 1.0940555 ]]
[[0.0469069  0.04777763 0.0930797  0.23165636 0.0074794  0.26273534]]
After normalisation, the minimal value of P and T is:  [0.05882353 0.        ]
After normalisation, the maximal value of P and T is:  [0.88235295 0.8       ]


In [43]:
pt_uniform = np.zeros(shape=(len(data), 2))
pt_uniform[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt_uniform[:, 1] = data["Temperature estimate random uniform"]

print("NaN values in PT: ", np.isnan(pt_uniform).any())

# train test split
PT02_biotite_composition_train, PT02_biotite_composition_val, PT02_pt_train, PT02_pt_val = train_test_split(biotite_composition, pt_uniform, test_size=VALIDATION_FRACTION, shuffle=True)

PT02_normalisation_biotite_composition = Normalization(axis=-1)
PT02_normalisation_biotite_composition.adapt(PT02_biotite_composition_train)

print(PT02_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(PT02_normalisation_biotite_composition.variance.numpy()))

PT02_pt_train_norm = scaling_pt(PT02_pt_train)
PT02_pt_val_norm = scaling_pt(PT02_pt_val)

print("After normalisation, the minimal value of P and T is: ", PT02_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", PT02_pt_train_norm.numpy().max(axis=0))

NaN values in PT:  False
[[2.7066655  0.12172261 1.711618   1.2324063  0.00855144 1.0851477 ]]
[[0.04751026 0.04804796 0.09292489 0.23100409 0.00746597 0.26331896]]
After normalisation, the minimal value of P and T is:  [0.00010304 0.00551514]
After normalisation, the maximal value of P and T is:  [0.92656535 0.89983726]


In [44]:
pt_ordered = np.zeros(shape=(len(data), 2))
pt_ordered[:, 0] = data["Pressure estimate random uniform"] * 1000 # convert to bar
pt_ordered[:, 1] = data["Temperature random ordered after Ti-in-Bt"]

print("NaN values in PT: ", np.isnan(pt_ordered).any())

# train test split
PT03_biotite_composition_train, PT03_biotite_composition_val, PT03_pt_train, PT03_pt_val = train_test_split(biotite_composition, pt_ordered, test_size=VALIDATION_FRACTION, shuffle=True)

PT03_normalisation_biotite_composition = Normalization(axis=-1)
PT03_normalisation_biotite_composition.adapt(PT03_biotite_composition_train)

print(PT03_normalisation_biotite_composition.mean.numpy())
print(np.sqrt(PT03_normalisation_biotite_composition.variance.numpy()))

PT03_pt_train_norm = scaling_pt(PT03_pt_train)
PT03_pt_val_norm = scaling_pt(PT03_pt_val)

print("After normalisation, the minimal value of P and T is: ", PT03_pt_train_norm.numpy().min(axis=0))
print("After normalisation, the maximal value of P and T is: ", PT03_pt_train_norm.numpy().max(axis=0))

NaN values in PT:  False
[[2.7068224  0.12022458 1.7133725  1.2351985  0.00839607 1.0837982 ]]
[[0.04753263 0.04760919 0.09376033 0.22823307 0.00730375 0.25924537]]
After normalisation, the minimal value of P and T is:  [0.00010304 0.00066034]
After normalisation, the maximal value of P and T is:  [0.940462  0.8988128]


## **Test PT 01:** Central estimate of *P* and *T* for the corresponding MAS / Zone

In [45]:
CALLBACKS_CENTRAL = [CSVLogger("Features_PTcentral.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [46]:
OPT = Adam(lr_schedule)

model_central = Sequential()
model_central.add(PT01_normalisation_biotite_composition)
model_central.add(Dense(16, activation="relu"))
model_central.add(Dense(2))

model_central.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
model_central.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_7 (Normaliza  (None, 6)                 13        
 tion)                                                           
                                                                 
 dense_14 (Dense)            (None, 16)                112       
                                                                 
 dense_15 (Dense)            (None, 2)                 34        
                                                                 
Total params: 159 (640.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


In [47]:
history = model_central.fit(PT01_biotite_composition_train, PT01_pt_train_norm,
                       batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                       validation_data=[PT01_biotite_composition_val, PT01_pt_val_norm],
                       callbacks=CALLBACKS_CENTRAL, verbose=False)

model_central.save(Path("saved_models", "model_feature_PT_central"))

INFO:tensorflow:Assets written to: saved_models\model_feature_PT_central\assets


INFO:tensorflow:Assets written to: saved_models\model_feature_PT_central\assets


## **Test PT 02:** Random sampling form a uniform distribution of *P* and *T* within P-/T-uppper and lower bounds for the corresponding MAS / Zone

In [48]:
CALLBACKS_UNIFORM = [CSVLogger("Features_PTuniform.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [49]:
OPT = Adam(lr_schedule)

model_uniform = Sequential()
model_uniform.add(PT02_normalisation_biotite_composition)
model_uniform.add(Dense(16, activation="relu"))
model_uniform.add(Dense(2))

model_uniform.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
model_uniform.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_8 (Normaliza  (None, 6)                 13        
 tion)                                                           
                                                                 
 dense_16 (Dense)            (None, 16)                112       
                                                                 
 dense_17 (Dense)            (None, 2)                 34        
                                                                 
Total params: 159 (640.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


In [50]:
history = model_uniform.fit(PT02_biotite_composition_train, PT02_pt_train_norm,
                       batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                       validation_data=[PT02_biotite_composition_val, PT02_pt_val_norm],
                       callbacks=CALLBACKS_UNIFORM, verbose=False)

model_uniform.save(Path("saved_models", "model_feature_PT_uniform"))

INFO:tensorflow:Assets written to: saved_models\model_feature_PT_uniform\assets


INFO:tensorflow:Assets written to: saved_models\model_feature_PT_uniform\assets


## **Test PT 03:** Random sampling from a uniform distribution of *P* and *T* within the P-/T-upper and lower bounds and additional ordering of the T estimates after Ti-in-biotite thermometry (Henry, 2005).

In [51]:
CALLBACKS_ORDERED = [CSVLogger("Features_PTordered.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [52]:
OPT = Adam(lr_schedule)

model_ordered = Sequential()
model_ordered.add(PT03_normalisation_biotite_composition)
model_ordered.add(Dense(16, activation="relu"))
model_ordered.add(Dense(2))

model_ordered.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
model_ordered.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_9 (Normaliza  (None, 6)                 13        
 tion)                                                           
                                                                 
 dense_18 (Dense)            (None, 16)                112       
                                                                 
 dense_19 (Dense)            (None, 2)                 34        
                                                                 
Total params: 159 (640.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


In [53]:
history = model_ordered.fit(PT03_biotite_composition_train, PT03_pt_train_norm,
                       batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                       validation_data=[PT03_biotite_composition_val, PT03_pt_val_norm],
                       callbacks=CALLBACKS_ORDERED, verbose=False)

model_ordered.save(Path("saved_models", "model_feature_PT_ordered"))

INFO:tensorflow:Assets written to: saved_models\model_feature_PT_ordered\assets


INFO:tensorflow:Assets written to: saved_models\model_feature_PT_ordered\assets
